<a href="https://colab.research.google.com/github/Kzis/mrt-is/blob/master/Prep_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv" , nrows=10000)
df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv")

In [6]:
df.shape

(17240769, 24)

In [7]:
df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


In [8]:
def get_data_by_month(month):
  return df.loc[df['ThaiMonthName'] == month]

def get_transaction_df(month_df):
  df_month_transaction_date_sum = month_df.TransactionDate.value_counts().rename_axis('TransactionDate').reset_index(name='TransactionCounts')
  df_month_transaction_date_sum.sort_values('TransactionDate')
  return df_month_transaction_date_sum

In [9]:
def get_entry_total_station_by_date(month_df , month_transaction_df , col_name):
  date_list = month_transaction_df['TransactionDate'].tolist()
  col_list = month_df[col_name].unique().tolist()
  output = []
  for date in date_list:
    # print("Process : {}".format(date))
    for col_val in col_list:
      lst = []
      subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
      column_count = subset_df.value_counts()
      lst.extend([date,"Entry", col_val, column_count.index[0]])
      output.append(lst)
      # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
  # print(output)
  df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
  return df  

def get_exit_total_station_by_date(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    output = []
    for date in date_list:
      # print("Process : {}".format(date))
      for col_val in col_list:
        lst = []
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.extend([date,"Exit", col_val, column_count.index[0]])
        output.append(lst)
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
    # print(output)
    df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    return df  

In [10]:
jan_df = get_data_by_month("มกราคม")
jan_df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


In [11]:
transaction_df = get_transaction_df(jan_df)
transaction_df.head()

,TransactionDate,TransactionCounts
0,2017-01-25,37135
1,2017-01-26,36485
2,2017-01-13,36171
3,2017-01-24,36050
4,2017-01-20,35898


# Prep ข้อมูล เข้าและออกทั้งหมดของแต่ละสถานี

In [12]:
 entry_df = get_entry_total_station_by_date(jan_df , transaction_df , "StationName")
 entry_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Entry,Sam Yan,1328
1,2017-01-25,Entry,Si Lom,2865
2,2017-01-25,Entry,Lumphini,1448
3,2017-01-25,Entry,Khlong Toei,308
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400
...,...,...,...,...
553,2017-01-01,Entry,Phahon Yothin,352
554,2017-01-01,Entry,Chatuchak Park,359
555,2017-01-01,Entry,Kamphaeng Phet,233
556,2017-01-01,Entry,Bang Sue,186


In [13]:
exit_df = get_exit_total_station_by_date(jan_df , transaction_df , "Right_StationName")
exit_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Exit,Hua Lamphong,1338
1,2017-01-25,Exit,Sam Yan,1299
2,2017-01-25,Exit,Si Lom,2720
3,2017-01-25,Exit,Lumphini,1621
4,2017-01-25,Exit,Khlong Toei,289
...,...,...,...,...
553,2017-01-01,Exit,Lat Phrao,309
554,2017-01-01,Exit,Phahon Yothin,342
555,2017-01-01,Exit,Chatuchak Park,250
556,2017-01-01,Exit,Kamphaeng Phet,151


In [14]:
jan_transaction_df = entry_df.append(exit_df) 
jan_transaction_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Entry,Sam Yan,1328
1,2017-01-25,Entry,Si Lom,2865
2,2017-01-25,Entry,Lumphini,1448
3,2017-01-25,Entry,Khlong Toei,308
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400
...,...,...,...,...
553,2017-01-01,Exit,Lat Phrao,309
554,2017-01-01,Exit,Phahon Yothin,342
555,2017-01-01,Exit,Chatuchak Park,250
556,2017-01-01,Exit,Kamphaeng Phet,151


In [42]:
jan_date_all_df = jan_transaction_df['TransactionDate']
jan_date_all_df

0      2017-01-25
1      2017-01-25
2      2017-01-25
3      2017-01-25
4      2017-01-25
          ...    
553    2017-01-01
554    2017-01-01
555    2017-01-01
556    2017-01-01
557    2017-01-01
Name: TransactionDate, Length: 1116, dtype: object

# Prep ข้อมูลประเภทของวัน

In [43]:
def get_day_type_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output
    # df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    # return df

In [44]:
day_type_list = get_day_type_list(jan_df , transaction_df , "DayType")

['WE', 'PH', 'WD']
[['2017-01-25', 0, 0, 37135], ['2017-01-26', 0, 0, 36485], ['2017-01-13', 0, 0, 36171], ['2017-01-24', 0, 0, 36050], ['2017-01-20', 0, 0, 35898], ['2017-01-11', 0, 0, 35831], ['2017-01-18', 0, 0, 35575], ['2017-01-10', 0, 0, 35343], ['2017-01-31', 0, 0, 35253], ['2017-01-19', 0, 0, 35208], ['2017-01-17', 0, 0, 35107], ['2017-01-09', 0, 0, 34666], ['2017-01-23', 0, 0, 34660], ['2017-01-06', 0, 0, 33946], ['2017-01-27', 0, 0, 33717], ['2017-01-16', 0, 0, 33200], ['2017-01-05', 0, 0, 33129], ['2017-01-30', 0, 0, 32814], ['2017-01-12', 0, 0, 31840], ['2017-01-04', 0, 0, 31479], ['2017-01-28', 16397, 0, 0], ['2017-01-21', 16184, 0, 0], ['2017-01-14', 16018, 0, 0], ['2017-01-07', 15156, 0, 0], ['2017-01-29', 13026, 0, 0], ['2017-01-03', 0, 12134, 0], ['2017-01-22', 12071, 0, 0], ['2017-01-15', 12068, 0, 0], ['2017-01-08', 11123, 0, 0], ['2017-01-02', 0, 7217, 0], ['2017-01-01', 5045, 0, 0]]


In [64]:
day_type_temp = pd.DataFrame(day_type_list, columns =['Date','WE','PH','WD'])
day_type_temp['DayType'] = ( (day_type_temp["WE"] != 0) * 1 ) + ( (day_type_temp["PH"] != 0) * 2 ) + ( (day_type_temp["WD"] != 0) * 3 )
day_type_temp.loc[day_type_temp['DayType'] == 1, 'DayType'] = "WE"
day_type_temp.loc[day_type_temp['DayType'] == 2, 'DayType'] = "PH"
day_type_temp.loc[day_type_temp['DayType'] == 3, 'DayType'] = "WD"
day_type_temp.drop('WE', axis=1, inplace=True)
day_type_temp.drop('PH', axis=1, inplace=True)
day_type_temp.drop('WD', axis=1, inplace=True)
day_type_temp

,Date,DayType
0,2017-01-25,WD
1,2017-01-26,WD
2,2017-01-13,WD
3,2017-01-24,WD
4,2017-01-20,WD
5,2017-01-11,WD
6,2017-01-18,WD
7,2017-01-10,WD
8,2017-01-31,WD
9,2017-01-19,WD


In [ ]:
jan_transaction_df

In [47]:
df_day_type = pd.DataFrame(columns=['DayType'])
df_day_type

,DayType


# Export Data 

In [ ]:
jan_transaction_df.to_excel (r'Jan_Summary.xlsx', index = False, header=True)